In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


In [2]:
from transformers import VisionEncoderDecoderModel, AutoFeatureExtractor, AutoTokenizer

In [3]:
img_encoder = 'google/vit-base-patch16-224-in21k'
text_decoder = 'gpt2'
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(img_encoder, text_decoder)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.3.crossattention.c_proj.weight', 'h.8.ln_cross_attn.bias', 'h.10.crossattention.c_attn.bias', 'h.4.crossattention.c_proj.bias', 'h.0.ln_cross_attn.bias', 'h.3.crossattention.c_attn.weight', 'h.0.crossattention.c_attn.bias', 'h.4.crossattention.q_attn.bias', 'h.6.crossattention.c_attn.weight', 'h.1.ln_cross_attn.bias', 'h.5.crossattention.c_proj.bias', 'h.6.crossattention.q_attn.weight', 'h.2.crossattention.c_attn.weight', 'h.9.crossattention.c_attn.weight', 'h.5.crossattention.q_attn.bias', 'h.11.ln_cross_attn.weight', 'h.1.crossattention.c_attn.weight', 'h.2.ln_cross_attn.bias', 'h.6.ln_cross_attn.weight', 'h.7.crossattention.c_proj.bias', 'h.11.crossattention.q_attn.weight', 'h.2.crossattention.q_attn.weight', 'h.8.crossattention.c_proj.bias', 'h.6.crossattention.c_proj.weight', 'h.8.crossattention.c_attn.weight', 'h.0.crossattention.q_attn.bias', 'h.6.crossattention.

In [4]:
feature_extractor = AutoFeatureExtractor.from_pretrained(img_encoder)
tokenizer = AutoTokenizer.from_pretrained(text_decoder)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
tokenizer.pad_token = tokenizer.eos_token

model.config.eos_token_id = tokenizer.eos_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [7]:
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
from PIL import Image
import PIL
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib

USER_AGENT = get_datasets_user_agent()

In [8]:
dataset = load_dataset("conceptual_captions")
dataset

Generating train split:   0%|          | 0/3318333 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15840 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image_url', 'caption'],
        num_rows: 3318333
    })
    validation: Dataset({
        features: ['image_url', 'caption'],
        num_rows: 15840
    })
})

In [9]:
def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image


def fetch_images(batch, num_threads, timeout=None, retries=0):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image"] = list(executor.map(fetch_single_image_with_args, batch["image_url"]))
    return batch


In [ ]:
num_threads = 20
dataset = dataset['train'].map(fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})
dataset = dataset['validation'].map(fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

In [11]:
def tokenization(caption, max_len):
    labels = tokenizer(caption,padding='max_length', max_length=max_len).input_ids
    return labels
def feature(img_paths):
    images = [Image.open(img) for img in img_paths]
    encoder_inputs = feature_extractor(images=images, return_tensor='np')
    return encoder_inputs.pixel_values
def preprocess(examples, max_len):
    caption = examples['caption']
    img_paths = examples['image_url']

    data = {}
    data['labels'] = tokenization(caption, max_len)
    data['pixel_values'] = feature(img_paths)
    return data


In [ ]:
proc_ds = dataset.map(
    function=preprocess,
    fn_kwargs={'max_len':200},
    batched=True
)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir = './img_caption_feat',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy='epoch',
    logging_steps=100,
    )

In [ ]:
import evaluate
import numpy as np
import nltk

metric = evaluate.load("rouge")

ignore_pad_token_for_loss = True

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if ignore_pad_token_for_loss:.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
                                                     decoded_labels)

    result = metric.compute(predictions=decoded_preds,
                            references=decoded_labels,
                            use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [ ]:
from transformers import default_data_collator

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=proc_ds['train'],
    eval_dataset = proc_ds['validation'],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train